## Load data

In [ ]:
# 24.02.24
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from sklearn.model_selection import train_test_split

# Load data
train = pd.read_csv("set/train.csv")
test = pd.read_csv("set/test.csv")
sample_sub = pd.read_csv("set/sample_solution.csv", index_col="Id")

# Preprocess data
X_train = np.array(train.iloc[:, 2:]).reshape(-1, 28, 28, 1).astype(np.float32) / 255.0
X_test = np.array(test.iloc[:, 1:]).reshape(-1, 28, 28, 1).astype(np.float32) / 255.0
y_col = pd.get_dummies(train['Categoria']).columns  
y_train = pd.get_dummies(train['Categoria']).to_numpy()

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


## Test 1

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal', input_shape=(28, 28, 1)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Dropout(0.25),
    
    layers.Conv2D(64, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Dropout(0.3),
    
    layers.Conv2D(128, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Dropout(0.4),            # --> BEST: 0.9395

    layers.Flatten(),
    layers.Dense(256, activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

## Test 2

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal', input_shape=(28, 28, 1)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Dropout(0.25),
    
    layers.Conv2D(64, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Dropout(0.32),
    
    layers.Conv2D(128, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Dropout(0.42),       # BEST -> 9422

    layers.Flatten(),
    layers.Dense(256, activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Dropout(0.51),
    layers.Dense(10, activation='softmax')
])

### Compile

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Fit

In [ ]:

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = LearningRateScheduler(lambda epoch, lr: lr * 0.9 if epoch % 10 == 0 and epoch != 0 else lr)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True)

# Train model
#history = model.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_val, y_val), callbacks=[early_stopping, lr_scheduler, model_checkpoint])
history = model.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_val, y_val))

### Prediction

In [ ]:
# Load the best model
#model.load_weights('best_model.h5')

# Predictions
prediction = model.predict(X_test)
pred_df = pd.DataFrame(prediction,columns=y_col,index=sample_sub.index)
prediction_col = pred_df.idxmax(axis=1)
sample_sub.Categoria = prediction_col

sample_sub.to_csv("submission_2.csv",index=True)